In [1]:
%pip install pyyaml
%pip install pyMongo

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import yaml
import pymongo
from urllib.parse import quote_plus as quote

In [3]:
# Подключимся к нашей коллекции

with open('../env/creds.yaml', 'r') as file:
    creds_dict = yaml.safe_load(file)
    
url = 'mongodb://{user}:{pw}@{hosts}/?{rs}&authSource={auth_src}&{am}&tls=true&tlsCAFile={cert_file}'.format(
    user=creds_dict['username'],
    pw=quote(creds_dict['password']),
    hosts=creds_dict['host'],
    rs='replicaSet=rs01',
    auth_src=creds_dict['database'],
    am='authMechanism=DEFAULT',
    cert_file='../env/root.crt'
    )

dbs = pymongo.MongoClient(url)

db = dbs[creds_dict['database']]

collection = db['initial_dataset']

In [4]:
# Пример aggregation pipeline - чтобы достать только нужные поля из Mongo
pipeline = [{ "$unwind" : "$data_result.boxes" },
            {"$project":{
                "mass": "$data_result.boxes.mass",
                "size": "$data_result.boxes.size",
                "stacking": "$data_result.boxes.stacking",
                "turnover": "$data_result.boxes.turnover",
                "stacking_limit": "$data_result.boxes.stacking_limit",
                "loading_size": "$data_result.cargo_space.loading_size",
                "density_percent": "$data_result.cargo_space.calculation_info.density_percent",
                "filling_space_percent": "$data_result.cargo_space.calculation_info.filling_space_percent",
                }},
            {"$group": {"_id": "$_id",
                        "loading_size": {"$first":"$loading_size"},
                        "density_percent": {"$first":"$density_percent"},
                        "filling_space_percent": {"$first":"$filling_space_percent"},
                        "boxes": {
                            "$push":  {
                                "mass": "$mass",
                                "size": "$size",
                                "stacking": "$stacking",
                                "turnover": "$turnover",
                                "stacking_limit": "$stacking_limit"                               
                            }
                            
                        }
            }
            }
            ]
result = collection.aggregate(pipeline)
#item = result.next()
#print(item)
dataset = [i for i in result]

In [5]:
len(dataset)

628

In [6]:
type(dataset)

list

In [7]:
dataset[0]

{'_id': ObjectId('6427400ea78245bedd47ac48'),
 'loading_size': {'width': 2352.0, 'height': 2393.0, 'length': 5898.0},
 'density_percent': 75.40325,
 'filling_space_percent': 51.77125,
 'boxes': [{'mass': 705.0,
   'size': {'width': 548.0, 'height': 587.0, 'length': 1300.0},
   'stacking': True,
   'turnover': False,
   'stacking_limit': 0.0},
  {'mass': 705.0,
   'size': {'width': 548.0, 'height': 587.0, 'length': 1300.0},
   'stacking': True,
   'turnover': False,
   'stacking_limit': 0.0},
  {'mass': 705.0,
   'size': {'width': 548.0, 'height': 587.0, 'length': 1300.0},
   'stacking': True,
   'turnover': False,
   'stacking_limit': 0.0},
  {'mass': 705.0,
   'size': {'width': 548.0, 'height': 587.0, 'length': 1300.0},
   'stacking': True,
   'turnover': False,
   'stacking_limit': 0.0},
  {'mass': 705.0,
   'size': {'width': 548.0, 'height': 587.0, 'length': 1300.0},
   'stacking': True,
   'turnover': False,
   'stacking_limit': 0.0},
  {'mass': 705.0,
   'size': {'width': 548.0, '

In [8]:
target_array =  []
for data in dataset:
    target_array.append(data['filling_space_percent']/100)
    del data['filling_space_percent']
    del data['density_percent']

In [9]:
source_array = []
for data in dataset:
    source_array_1 = [[],[],[],[],[]]
    for box in data['boxes']:
        source_array_1[0].append(box['size']['length'])
        source_array_1[1].append(box['size']['width'])
        source_array_1[2].append(box['size']['height'])
        source_array_1[3].append(int(box['stacking']))
        source_array_1[4].append(int(box['turnover']))
    source_array.append(source_array_1)

In [10]:
length = max(map(len, source_array))
for arr in source_array:
    if (max(map(len, arr)) > length):
        length = max(map(len, arr))
length

2849

In [11]:
%pip install numpy
import numpy as np

Note: you may need to restart the kernel to use updated packages.


In [12]:
X = []
for arr in source_array:
    X.append(np.array([xi+[0]*(length-len(xi)) for xi in arr]))

source = np.array(X)

In [13]:
%pip install -U scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [14]:
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [15]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense, Flatten, Reshape
import matplotlib.pyplot as plt

In [22]:
X = np.asarray(source).astype(float)
Y = np.asarray(target_array).astype(float)

normalizer1 = tf.keras.layers.experimental.preprocessing.Normalization(axis=None)
normalizer1.adapt(X)

print(X.shape)
print(Y.shape)

(628, 5, 2849)
(628,)


In [17]:
from sklearn.metrics import r2_score, mean_squared_error

In [29]:
%pip install tensorflow_probability==0.7.0

     -------------------------------------- 981.4/981.4 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: tensorflow_probability
    Found existing installation: tensorflow-probability 0.19.0
    Uninstalling tensorflow-probability-0.19.0:
      Successfully uninstalled tensorflow-probability-0.19.0
Note: you may need to restart the kernel to use updated packages.


In [33]:
inp = Input(shape=(5, 2849, 1))
x=inp
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1)




TypeError: Cannot iterate over a Tensor with unknown first dimension.

In [25]:
y_pred = model.predict(X_test).flatten()
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print("R^2 score: {:.4f}".format(r2))
print("MSE: {:.4f}".format(mse))


2/2 [==============================] - 0s 5ms/step
R^2 score: -1.4878
MSE: 0.1270
